In [ ]:
import os
# ─── your existing environment setup ────────────────────────
import env_config
# ─────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
from plancklens.utils import cli
from binner_sims import ffp10_binner_phiT
import thesis_plot_style as tps

# ─── CONFIG ────────────────────────────────────────────────────────────────
bin_type = "agr2"
sim_idxs = np.arange(60, 300)
Lmin, Lmax = 3, 100
Ls = np.arange(Lmin, Lmax+1)

# ─── PARAMETER FILES & SCENARIOS ──────────────────────────────────────────
import parfiles.noNoise.parfile                     as par_nn
# import parfiles.noNoise.parfile_delensed            as par_nn_del
import parfiles.noNoise.parfile_delensed_qest       as par_nn_qest
import parfiles.noNoise.parfile_delensed_PolQEST    as par_nn_polq
import parfiles.Noise.parfile               as par_n
# import parfiles.Noise.parfile_delensed      as par_n_del
import parfiles.Noise.parfile_delensed_qest as par_n_qest
import parfiles.Noise.parfile_delensed_PolQEST as par_n_polq

scenarios_noiseless = [
    (par_nn,    "MV: lensed",                  'p'),
    # (par_nn_del,"MV: delensed with input KLM", 'p'),
    (par_nn_qest, "MV: internally delensed with MV-QEST", 'p'),
    (par_nn_polq, "TT: internally delensed with Pol-QEST", 'ptt'),
]
scenarios_noisy = [
    (par_n,      "MV: lensed",                  'p'),
    # (par_n_del,  "MV: delensed with input KLM", 'p'),
    (par_n_qest,"MV: internally delensed with MV-QEST", 'p'),
    (par_n_polq,"TT: internally delensed with Pol-QEST", 'ptt'),
]

# ─── 1) Fiducial C_ℓ & bins ────────────────────────────────────────────────
b0    = ffp10_binner_phiT("p", par_nn, bin_type)
lmax  = b0.lmax
C_fid = b0.clpt_fid.copy()
fid_sq = C_fid[Ls]**2

# ─── 2) Compute σₗ per scenario & noise ────────────────────────────────────
sigma = {"noiseless": {}, "noisy": {}}
for noise_key, scen_list in [("noiseless", scenarios_noiseless),
                             ("noisy",     scenarios_noisy)]:
    for parfile, label, kphi in scen_list:
        b = ffp10_binner_phiT(kphi, parfile, bin_type)
        _, sig = b.get_cL_PHI_T_with_error(
            mc_sims=sim_idxs,
            binned=False,
            scaled=False
        )
        sigma[noise_key][label] = sig

# ─── 3) Load raw CL_sim (no scaling) ──────────────────────────────────────
CL_sim = {"noiseless": {}, "noisy": {}}
for noise_key, scen_list in [("noiseless", scenarios_noiseless),
                             ("noisy",     scenarios_noisy)]:
    for parfile, label, kphi in scen_list:
        arr = np.zeros((sim_idxs.size, lmax+1))
        resp = cli(parfile.qresp_dd.get_response(kphi, 'p')[:lmax+1])
        for ii, idx in enumerate(sim_idxs):
            cl = parfile.qcls_pt.get_sim_qcl(kphi, idx, lmax=lmax).copy()
            cl *= resp
            arr[ii] = cl
        CL_sim[noise_key][label] = arr

# ─── 4) Build weights & denominators ──────────────────────────────────────
w   = {"noiseless": {}, "noisy": {}}
den = {"noiseless": {}, "noisy": {}}

for noise_key in ["noiseless", "noisy"]:
    for label, sig in sigma[noise_key].items():
        seg = sig[Ls].copy()
        # guard against zeros
        seg[seg == 0] = np.finfo(float).eps
        w_arr = 1.0 / seg**2
        w[noise_key][label] = w_arr
        den[noise_key][label] = np.sum(fid_sq * w_arr)

# ─── 5) Estimator & compute A_vals + stats (no renormalization) ─────────
def estimate_A(cl_row, w_arr, denom):
    num = np.sum(cl_row[Ls] * C_fid[Ls] * w_arr)
    return num/denom

A_vals = {"noiseless": {}, "noisy": {}}
stats  = {"noiseless": {}, "noisy": {}}

for noise_key in ["noiseless", "noisy"]:
    for label in sigma[noise_key]:
        w_arr = w[noise_key][label]
        d     = den[noise_key][label]
        sims  = CL_sim[noise_key][label]
        A     = np.array([estimate_A(sims[i], w_arr, d) for i in range(sims.shape[0])])
        A_vals[noise_key][label] = A
        m, s = A.mean(), A.std(ddof=1)
        stats[noise_key][label] = (m, s)

# ─── 6) Print out your raw means & σ’s ─────────────────────────────────────
for noise_key in stats:
    print(f"\n=== {noise_key.upper()} ===")
    for label, (m, s) in stats[noise_key].items():
        print(f"{label:35s}  mean = {m:.3f}   σ = {s:.3f}")

from matplotlib.lines import Line2D

# ─── 7) Plot histograms with mean±σ in legend ─────────────────────────────
abbr = {
    "MV: lensed":                               "Lensed",
    "MV: internally delensed with MV-QEST":     "MV-QE",
    "TT: internally delensed with Pol-QEST":    "Pol-QE",
}

tps.apply_style("single")
for noise_key in ["noiseless", "noisy"]:
    fname = f"fig_AphiT_hist_{noise_key}.png"
    with tps.context_figure(filename=fname) as (fig, ax):
        # plot histograms
        for color, label in zip(tps.CB_PALETTE, A_vals[noise_key]):
            data = A_vals[noise_key][label]
            ax.hist(
                data,
                bins=30, density=True, histtype="step",
                color=color
            )

        ax.set(
            xlabel=r"$A_{\phi T}$",
            ylabel="Probability Density",
            title=f"Distribution of $A_{{\phi T}}$ ({noise_key})"
        )

        # build custom legend handles + labels
        handles, labels = [], []
        for color, label in zip(tps.CB_PALETTE, A_vals[noise_key]):
            m, s = stats[noise_key][label]
            name = abbr.get(label, label)
            h = Line2D([0], [0], color=color, lw=1.8)
            handles.append(h)
            labels.append(f"{name}: mean={m:.3f}, σ={s:.3f}")

        ax.legend(handles, labels, loc="upper right", fontsize=9, frameon=False)


Using lenspyx alm2map

=== NOISELESS ===
MV: lensed                           mean = 0.988   σ = 0.198
MV: internally delensed with MV-QEST  mean = 0.175   σ = 0.043
TT: internally delensed with Pol-QEST  mean = 0.423   σ = 0.208

=== NOISY ===
MV: lensed                           mean = 0.962   σ = 0.298
MV: internally delensed with MV-QEST  mean = 0.470   σ = 0.169
TT: internally delensed with Pol-QEST  mean = 0.851   σ = 0.336


In [2]:
import numpy as np

# ─── assuming A_vals["noisy"][label] already exists ────────────────────────
batch_size = 16
n_batches  = 240 // batch_size   # = 15

sigma_of_sigma = {}
full_sigma     = {}

for label in A_vals["noiseless"]:
    arr = A_vals["noiseless"][label]
    # reshape into (15,16) batches
    batches = arr.reshape(n_batches, batch_size)
    # sigma per batch
    batch_sigmas = batches.std(axis=1, ddof=1)
    # full‐sample sigma
    sig_full = arr.std(ddof=1)
    # uncertainty on that sigma ≃ stddev of the batch sigmas
    sig_unc  = batch_sigmas.std(ddof=1)

    full_sigma[label]     = sig_full
    sigma_of_sigma[label] = sig_unc

    print(f"{label:35s}  σ = {sig_full:.3f}  ± {sig_unc:.3f}")

# ─── how significant is the drop from “MV: lensed” → “MV: internally delensed”? ───
σ1, δσ1 = full_sigma["MV: lensed"],     sigma_of_sigma["MV: lensed"]
σ2, δσ2 = full_sigma["MV: internally delensed with MV-QEST"], sigma_of_sigma["MV: internally delensed with MV-QEST"]
Δσ      = σ1 - σ2
σΔ      = np.sqrt(δσ1**2 + δσ2**2)
print()
print(f"Drop Δσ = {Δσ:.3f} ± {σΔ:.3f}  →  significance = {Δσ/σΔ:.1f} σ")

MV: lensed                           σ = 0.198  ± 0.026
MV: internally delensed with MV-QEST  σ = 0.043  ± 0.009
TT: internally delensed with Pol-QEST  σ = 0.208  ± 0.044

Drop Δσ = 0.155 ± 0.028  →  significance = 5.6 σ


In [16]:
import numpy as np

# ─── assuming A_vals["noisy"][label] already exists ────────────────────────
batch_size = 16
n_batches  = 240 // batch_size   # = 15

sigma_of_sigma = {}
full_sigma     = {}

for label in A_vals["noisy"]:
    arr = A_vals["noisy"][label]
    # reshape into (15,16) batches
    batches = arr.reshape(n_batches, batch_size)
    # sigma per batch
    batch_sigmas = batches.std(axis=1, ddof=1)
    # full‐sample sigma
    sig_full = arr.std(ddof=1)
    # uncertainty on that sigma ≃ stddev of the batch sigmas
    sig_unc  = batch_sigmas.std(ddof=1)

    full_sigma[label]     = sig_full
    sigma_of_sigma[label] = sig_unc

    print(f"{label:35s}  σ = {sig_full:.3f}  ± {sig_unc:.3f}")

# ─── how significant is the drop from “MV: lensed” → “MV: internally delensed”? ───
σ1, δσ1 = full_sigma["MV: lensed"],     sigma_of_sigma["MV: lensed"]
σ2, δσ2 = full_sigma["MV: internally delensed with MV-QEST"], sigma_of_sigma["MV: internally delensed with MV-QEST"]
Δσ      = σ1 - σ2
σΔ      = np.sqrt(δσ1**2 + δσ2**2)
print()
print(f"Drop Δσ = {Δσ:.3f} ± {σΔ:.3f}  →  significance = {Δσ/σΔ:.1f} σ")


MV: lensed                           σ = 0.298  ± 0.033
MV: internally delensed with MV-QEST  σ = 0.169  ± 0.026
TT: internally delensed with Pol-QEST  σ = 0.336  ± 0.053

Drop Δσ = 0.129 ± 0.042  →  significance = 3.1 σ


In [ ]:
import os
# ─── your existing environment setup ────────────────────────
import env_config
# ─────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
from plancklens.utils import cli
from binner_sims import ffp10_binner_phiT
import thesis_plot_style as tps
import seaborn as sns

# ─── CONFIG ────────────────────────────────────────────────────────────────
bin_type = "agr2"
sim_idxs = np.arange(60, 300)
Lmin, Lmax = 3, 100
Ls = np.arange(Lmin, Lmax+1)

# ─── PARAMETER FILES & SCENARIOS ──────────────────────────────────────────
# import parfiles.noNoise.parfile                     as par_nn
# import parfiles.noNoise.parfile_delensed            as par_nn_del
# import parfiles.noNoise.parfile_delensed_qest       as par_nn_qest
# import parfiles.noNoise.parfile_delensed_PolQEST    as par_nn_polq
import parfiles.Noise.parfile               as par_n
# import parfiles.Noise.parfile_delensed      as par_n_del
import parfiles.Noise.parfile_delensed_qest as par_n_qest
import parfiles.Noise.parfile_delensed_PolQEST as par_n_polq

scenarios_noiseless = [
    # (par_nn,    "MV: lensed",                  'p'),
    # (par_nn_del,"MV: delensed with input KLM", 'p'),
    # (par_nn_qest, "MV: internally delensed with MV-QEST", 'p'),
    # (par_nn_polq, "TT: internally delensed with Pol-QEST", 'ptt'),
]
scenarios_noisy = [
    (par_n,      "MV: lensed",                  'p'),
    # (par_n_del,  "MV: delensed with input KLM", 'p'),
    (par_n_qest,"MV: internally delensed with MV-QEST", 'p'),
    (par_n_polq,"TT: internally delensed with Pol-QEST", 'ptt'),
]

# ─── 1) Fiducial C_ℓ & bins ────────────────────────────────────────────────
b0    = ffp10_binner_phiT("p", par_n, bin_type)
lmax  = b0.lmax
C_fid = b0.clpt_fid.copy()
fid_sq = C_fid[Ls]**2

# ─── 2) Compute σₗ per scenario & noise ────────────────────────────────────
sigma = {"noiseless": {}, "noisy": {}}
for noise_key, scen_list in [("noiseless", scenarios_noiseless),
                             ("noisy",     scenarios_noisy)]:
    for parfile, label, kphi in scen_list:
        b = ffp10_binner_phiT(kphi, parfile, bin_type)
        _, sig = b.get_cL_PHI_T_with_error(
            mc_sims=sim_idxs,
            binned=False,
            scaled=False
        )
        sigma[noise_key][label] = sig

# ─── 3) Load raw CL_sim (no scaling) ──────────────────────────────────────
CL_sim = {"noiseless": {}, "noisy": {}}
for noise_key, scen_list in [("noiseless", scenarios_noiseless),
                             ("noisy",     scenarios_noisy)]:
    for parfile, label, kphi in scen_list:
        arr = np.zeros((sim_idxs.size, lmax+1))
        resp = cli(parfile.qresp_dd.get_response(kphi, 'p')[:lmax+1])
        for ii, idx in enumerate(sim_idxs):
            cl = parfile.qcls_pt.get_sim_qcl(kphi, idx, lmax=lmax).copy()
            cl *= resp
            arr[ii] = cl
        CL_sim[noise_key][label] = arr

# ─── 4) Build weights & denominators ──────────────────────────────────────
w   = {"noiseless": {}, "noisy": {}}
den = {"noiseless": {}, "noisy": {}}

for noise_key in ["noiseless", "noisy"]:
    for label, sig in sigma[noise_key].items():
        seg = sig[Ls].copy()
        # guard against zeros
        seg[seg == 0] = np.finfo(float).eps
        w_arr = 1.0 / seg**2
        w[noise_key][label] = w_arr
        den[noise_key][label] = np.sum(fid_sq * w_arr)

# ─── 5) Estimator & compute A_vals + stats (no renormalization) ─────────
def estimate_A(cl_row, w_arr, denom):
    num = np.sum(cl_row[Ls] * C_fid[Ls] * w_arr)
    return num/denom

A_vals = {"noiseless": {}, "noisy": {}}
stats  = {"noiseless": {}, "noisy": {}}

for noise_key in ["noiseless", "noisy"]:
    for label in sigma[noise_key]:
        w_arr = w[noise_key][label]
        d     = den[noise_key][label]
        sims  = CL_sim[noise_key][label]
        A     = np.array([estimate_A(sims[i], w_arr, d) for i in range(sims.shape[0])])
        A_vals[noise_key][label] = A
        m, s = A.mean(), A.std(ddof=1)
        stats[noise_key][label] = (m, s)

# ─── after you’ve computed A_vals, stats ─────────────────────────────────
# Compute the Planck‐mission A_data for each noisy scenario:
A_data = {}
for parfile, label, kphi in scenarios_noisy:
    # 1) fetch the data spectrum:
    cl_data = parfile.qcls_pt.get_sim_qcl(kphi, -1, lmax=lmax).copy()
    # 2) apply the same response filter:
    resp    = cli(parfile.qresp_dd.get_response(kphi, 'p')[:lmax+1])
    cl_data *= resp
    # 3) estimate A using that scenario’s weights & denom:
    w_arr = w["noisy"][label]
    denom = den["noisy"][label]
    A_data[label] = estimate_A(cl_data, w_arr, denom)

print("\nPlanck data A_phiT (noisy scenarios):")
for label, A in A_data.items():
    print(f"  {label:35s}  A_data = {A:.3f}")

import seaborn as sns
from matplotlib.lines import Line2D

# … (all the code above up through computing A_vals and A_data) …

order = list(A_vals["noisy"].keys())
abbr = {
    "MV: lensed":                    "Lensed",
    "MV: internally delensed with MV-QEST": "MV-QE",
    "TT: internally delensed with Pol-QEST": "Pol-QE",
}

tps.apply_style("single")
fig, ax = plt.subplots(figsize=(5, 4))

# plot the KDEs + dashed data lines
for color, label in zip(tps.CB_PALETTE, order):
    sns.kdeplot(
        A_vals["noisy"][label],
        ax=ax,
        color=color,
        lw=2,
        alpha=0.8
    )
    ax.axvline(
        A_data[label],
        color=color,
        linestyle="--",
        lw=2
    )

ax.set(
    xlabel=r"$A_{\phi T}$",
    ylabel="Density",
    title="KDE of $A_{\\phi T}$ (noisy) with Planck mission data"
)

# build custom legend handles for the Planck-data values
handles = []
labels = []
for color, label in zip(tps.CB_PALETTE, order):
    name = abbr[label]
    val  = A_data[label]
    # use a dashed line handle (matching the vline style)
    h = Line2D([0], [0], color=color, linestyle="--", lw=2)
    handles.append(h)
    labels.append(rf"{name}: $\widehat{{A}}_{{\phi T}}$ = {val:.3f}")

ax.legend(handles, labels, loc="upper right", frameon=False, fontsize=9)

# save
tps.savefig(fig, "fig_AphiT_kde_only.png")
plt.close(fig)



Planck data A_phiT (noisy scenarios):
  MV: lensed                           A_data = 0.711
  MV: internally delensed with MV-QEST  A_data = 0.504
  TT: internally delensed with Pol-QEST  A_data = 0.943


In [ ]:
import os
# ─── your existing environment setup ────────────────────────
import env_config
# ─────────────────────────────────────────────────────────────


import os
import numpy as np
import matplotlib.pyplot as plt
from plancklens.utils import cli
from binner_sims import ffp10_binner_phiT
import thesis_plot_style as tps

# Environment
# Configuration
bin_type = "agr2"
sim_idxs = np.arange(60,300)
Lmin, Lmax = 3, 100

# Instantiate binner for fiducial
import parfiles.noNoise.parfile as par_nn
b0    = ffp10_binner_phiT("p", par_nn, bin_type)
lmax  = b0.lmax
ells  = np.arange(lmax+1)
C_fid = b0.clpt_fid.copy()
Ls    = np.arange(Lmin, Lmax+1)
fid_sq = C_fid[Ls]**2

# Scenarios definition
import parfiles.noNoise.parfile_delensed_qest       as par_nn_qest
import parfiles.noNoise.parfile_delensed_PolQEST    as par_nn_polq
import parfiles.Noise.parfile               as par_n
import parfiles.Noise.parfile_delensed_qest as par_n_qest
import parfiles.Noise.parfile_delensed_PolQEST as par_n_polq
scenarios = {
    "noiseless": [
        (par_nn,    "MV: lensed",                  'p'),
        (par_nn_qest, "MV: internally delensed with MV-QEST", 'p'),
        (par_nn_polq, "TT: internally delensed with Pol-QEST", 'ptt'),
    ],
    "noisy": [
        (par_n,      "MV: lensed",                  'p'),
        (par_n_qest,"MV: internally delensed with MV-QEST", 'p'),
        (par_n_polq,"TT: internally delensed with Pol-QEST", 'ptt'),
    ]
}

# 1) Compute null spectra
null_cl, null_err = {}, {}
for noise_key, specs in scenarios.items():
    null_cl[noise_key] = {}
    null_err[noise_key] = {}
    for parfile, label, kphi in specs:
        b = ffp10_binner_phiT(kphi, parfile, bin_type)
        cl_bar, cl_err = b.get_cL_PHI_T_with_error(
            mc_sims=sim_idxs,
            binned=False,
            scaled=False,
            cross_maps=True
        )
        null_cl[noise_key][label]  = cl_bar
        null_err[noise_key][label] = cl_err

# 2) Fit amplitude

def fit_amplitude(cl_arr, sigma_arr):
    eps = np.finfo(float).eps
    seg = sigma_arr[Ls].copy()
    seg[seg == 0] = eps
    w    = 1.0 / seg**2
    den  = np.sum(fid_sq * w)
    num  = np.sum(cl_arr[Ls] * C_fid[Ls] * w)
    A    = num/den if den>0 else 0.0
    sA   = np.sqrt(1.0/den) if den>0 else 0.0
    return A, sA

null_stats = {nk: {} for nk in scenarios}
for noise_key in scenarios:
    for _, label, _ in scenarios[noise_key]:
        A, sA = fit_amplitude(
            null_cl[noise_key][label],
            null_err[noise_key][label]
        )
        null_stats[noise_key][label] = (A, sA)

# 3) Plot with separate markers for noiseless/noisy
from matplotlib.lines import Line2D
abbr = {
    "MV: lensed":                               "Lensed",
    "MV: internally delensed with MV-QEST":     "MV-QE",
    "TT: internally delensed with Pol-QEST":    "Pol-QE",
}
markers = {"noiseless": 'o', "noisy": 's'}  # circle vs square

tps.apply_style("single")
fig, ax = plt.subplots(figsize=(5, 3.2))

pipelines = [lbl for _, lbl, _ in scenarios["noiseless"]]
y0 = np.arange(len(pipelines))
dy = 0.15

for i, label in enumerate(pipelines):
    for j, (noise_key, dx) in enumerate([("noiseless", -dy), ("noisy", +dy)]):
        A, sA = null_stats[noise_key][label]
        mkr = markers[noise_key]
        if sA>0:
            ax.errorbar(
                A, y0[i]+dx,
                xerr=sA,
                fmt=mkr,
                ms=6,
                color=tps.CB_PALETTE[i],
                capsize=4,
                label=None
            )
        else:
            ax.plot(
                A, y0[i]+dx,
                marker=mkr,
                ms=6,
                color=tps.CB_PALETTE[i],
                label=None
            )

# zero line
ax.axvline(0, color='k', linestyle='--', lw=1)

# Legend handles
handles = []
labels = []
# marker legend
for nk in ['noiseless','noisy']:
    h = Line2D([0],[0], marker=markers[nk], color='k', linestyle='None', markersize=6)
    handles.append(h)
    labels.append(nk.capitalize())
# pipeline legend
for i, label in enumerate(pipelines):
    h = Line2D([0],[0], marker='o', color=tps.CB_PALETTE[i], linestyle='None', markersize=6)
    handles.append(h)
    labels.append(abbr[label])

ax.set(
    yticks=y0,
    yticklabels=[abbr[lbl] for lbl in pipelines],
    xlabel=r"$\widehat{A}_{\phi T}^{\mathrm{null}}$",
    xlim=(-0.03, 0.03),
    title=r"Null test: cross-corr $\phi \times T_{\mathrm{other}}$"
)
ax.invert_yaxis()
ax.legend(handles, labels, loc='lower right', ncol=1, frameon=False)  # 1 column legend

tps.savefig(fig, "fig_AphiT_nulltest_forest.png")
plt.close(fig)
